In [1]:
import re
from collections import Counter

In [2]:
def words(text):
    return re.findall(r'\w+', text.lower())

In [24]:
Words = Counter(words(open('big.txt').read()))

In [25]:
len(Words)

32198

In [27]:
sum(Words.values())

1115585

In [28]:
Words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [11]:
def P(word, N = sum(Words.values())):
    return Words[word]/N

In [31]:
max(Words, key=P)

'the'

In [32]:
P('the')

0.07154004401278254

In [12]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [33]:
len(edits1('somthing'))

442

In [13]:
def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [35]:
len(set(edits2('somthing')))

90902

In [14]:
def known(words):
    return set(w for w in words if w in Words)

In [40]:
known(edits1('something'))

{'something'}

In [36]:
known(edits1('somthing'))

{'something', 'soothing'}

In [37]:
known(edits2('somthing'))

{'loathing',
 'nothing',
 'scathing',
 'seething',
 'smoothing',
 'something',
 'soothing',
 'sorting'}

In [39]:
known(edits2('something'))

{'seething', 'smoothing', 'something', 'soothing'}

In [15]:
def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

In [16]:
def correction(word):
    return max(candidates(word), key=P)

In [17]:
correction('hungre')

'hungry'

In [18]:
correction('korrectud')

'corrected'

In [19]:
correction('arrainged')

'arranged'

In [21]:
correction('peotry')

'poetry'

In [46]:
def tokens(text):
    return re.findall('[a-z]+', text.lower())

In [48]:
tokens('This is: A test, 1, 2, 3, this is.')

['this', 'is', 'a', 'test', 'this', 'is']

In [47]:
map(correction, tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?'))

In [53]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correction(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [54]:
correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes?')

'Spelling Errors IN something. Whatever; unusual mistakes?'

In [55]:
correct_text('Audiance sayzs: tumblr ...')

'Audience says: tumbler ...'

## Evaluation

In [41]:
import time

In [42]:
def spelltest(tests):
    start = time.clock()
    good, unknown = 0,0
    n = len(tests)
    
    for right, wrong in tests:
        w = correction(wrong)
        good += (w==right)
        if w != right:
            unknown += (right not in Words)
    
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))

In [43]:
def Testset(lines):
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

In [44]:
spelltest(Testset(open('spell-testset.txt')))

75% of 270 correct (6% unknown) at 39 words per second 
